## Detector de **SPAM**

### Objetivo do projeto

Este projeto tem o objetivo de mostrar como o **Multilayer Perceptron** pode ser utilizado para a detecção de SPAM. Embora o projeto esteja em Língua Portuguesa do Brasil, a base de SMS utilizada foi [encontrada no Kaggle](https://www.kaggle.com/code/dhgupta/bag-of-words-model/input), em Língua Inglesa, com o original disponibilizado em `data/spam.csv`.

### Análise dos dados

O primeiro passo em um projeto dessa natureza é analisar os dados. Para tanto, faremos uma leitura da base original e vamos contar o número palavras e frases. Ao final dessa tarefa, vamos decidir por executar ou não algum pré-processamento.

In [ ]:
# Instalar pacotes necessários
%pip install -q pandas matplotlib seaborn wordcloud nltk

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Importar bibliotecas necessárias
import pandas as pd                    # Leitura e manipulação de dados em DataFrames
import matplotlib.pyplot as plt        # Gráficos e visualizações
import seaborn as sns                  # Gráficos estatísticos com uma estética aprimorada
from wordcloud import WordCloud        # Nuvens de palavras
import nltk                            # Processamento de linguagem natural (NLP)
from nltk.corpus import stopwords      # Lstas de stopwords (palavras irrelevantes)
from collections import Counter        # Frequência de elementos
import string                          # Manipulação de pontuação e caracteres especiais

In [ ]:
# 2. Leitura do arquivo
spam = pd.read_csv("data/spam.csv", encoding='utf-8')   # Codificação para evitar erros
spam = spam[['Label', 'EmailText']]                     # manter apenas colunas relevantes
spam.columns = ['label', 'sms']                         # renomeia para dar melhor sentido
print(f'Linhas da base SPAM original: {spam.shape}')    # Imprime dimensão da base
spam.head()                                             # Imprime cinco registros

Shape da base SPAM original: (5572, 2)


,label,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
# 3. Estatísticas básicas
df['word_count'] = df['text'].apply(lambda x: len(x.split()))
df['char_count'] = df['text'].apply(len)

print("Mensagens por categoria:")
print(df['label'].value_counts())

print("\nMédia de palavras por mensagem:")
print(df.groupby('label')['word_count'].mean())

print("\nTamanho da maior mensagem (em caracteres):")
print(df['char_count'].max())

print("\nMensagem mais longa:")
print(df.loc[df['char_count'].idxmax(), 'text'])

# 5. Visualização: histograma do comprimento das mensagens
plt.figure(figsize=(10,6))
sns.histplot(data=df, x='char_count', hue='label', bins=50, kde=True)
plt.title("Distribuição do tamanho das mensagens")
plt.xlabel("Número de caracteres")
plt.ylabel("Frequência")
plt.show()

# 6. Nuvem de palavras (WordCloud)
def generate_wordcloud(text, title):
    wc = WordCloud(width=800, height=400, background_color='white', stopwords='english').generate(text)
    plt.figure(figsize=(10,5))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.title(title, fontsize=16)
    plt.show()

# Juntar textos de cada categoria
spam_text = " ".join(df[df['label'] == 'spam']['text'])
ham_text = " ".join(df[df['label'] == 'ham']['text'])

generate_wordcloud(spam_text, "Nuvem de Palavras - SPAM")
generate_wordcloud(ham_text, "Nuvem de Palavras - HAM")

#### Análise de Frequência de Palavras

In [ ]:
import nltk
from nltk.corpus import stopwords
from collections import Counter
import string

# Baixar recursos do nltk (apenas na 1ª vez)
nltk.download('punkt')
nltk.download('stopwords')

# Função de limpeza e tokenização
def get_word_frequencies(text_series):
    stop_words = set(stopwords.words('english'))
    all_words = []
    for message in text_series:
        # Tokenizar
        words = nltk.word_tokenize(message.lower())
        # Remover pontuações e stopwords
        words = [word for word in words if word.isalpha() and word not in stop_words]
        all_words.extend(words)
    return Counter(all_words)

# Frequências de palavras
spam_freq = get_word_frequencies(df[df['label'] == 'spam']['text'])
ham_freq = get_word_frequencies(df[df['label'] == 'ham']['text'])

# Mostrar 20 palavras mais comuns
print("Top 20 palavras em SPAM:")
print(spam_freq.most_common(20))

print("\nTop 20 palavras em HAM:")
print(ham_freq.most_common(20))


####  Palavras mais comuns

In [ ]:
# Converter para DataFrame e plotar
def plot_common_words(word_counter, title, n=20):
    common_words = word_counter.most_common(n)
    words, counts = zip(*common_words)
    plt.figure(figsize=(10,6))
    sns.barplot(x=list(counts), y=list(words), palette="viridis")
    plt.title(title)
    plt.xlabel("Frequência")
    plt.ylabel("Palavra")
    plt.show()

plot_common_words(spam_freq, "Palavras Mais Comuns em SPAM")
plot_common_words(ham_freq, "Palavras Mais Comuns em HAM")
